In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import plotly.graph_objects as go

df = pd.read_csv("song_data.xls.csv")
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

features = df.drop(columns=["song_name", "song_popularity"])
scaler = MinMaxScaler()
X = scaler.fit_transform(features)

# applying knn
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')
knn_model.fit(X)


NearestNeighbors(metric='cosine', n_neighbors=6)

In [ ]:

def recommend_knn(song_name):
    song_name = song_name.strip().lower()

    if song_name not in df["song_name"].str.lower().values:
        print("\nSong not found. Try again.")
        return

    index = df[df["song_name"].str.lower() == song_name].index[0]
    original_song_name = df.iloc[index]["song_name"]

    distances, indices = knn_model.kneighbors([X[index]])
    rec_songs = [df.iloc[i]["song_name"] for i in indices[0][1:]]
    rec_distances = distances[0][1:]
    similarity_scores = [1 - d for d in rec_distances]

    print(f"\nBecause you like '{original_song_name}', you might also like:")
    for i, s in enumerate(rec_songs, 1):
        print(f"{i}. {s}")

    # Line Chart
    fig_line = go.Figure(go.Scatter(
    x=list(range(1, len(similarity_scores)+1)),
    y=similarity_scores,
    mode='lines+markers+text',
    line=dict(color='royalblue'),
    text=rec_songs,
    textposition='top center'
    ))
    fig_line.update_layout(
    title=f"Similarity Drop-off for Recommendations of '{original_song_name}'",
    xaxis_title="Recommendation Rank",
    yaxis_title="Similarity Score",
    template="plotly_white",
    height=450
    )
    fig_line.show()


user_song = input("Enter a song name: ")
recommend_knn(user_song)


Enter a song name: closer

Because you like 'ZEZE (feat. Travis Scott & Offset)', you might also like:
1. ZEZE (feat. Travis Scott & Offset)
2. No Scrubs
3. Doin 2 Much
4. Tipsy - Club Mix
5. Heard 'Em Say
